In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain.agents import Tool
from langchain.agents import AgentType
from langchain.memory import ConversationSummaryMemory, ChatMessageHistory
from langchain.chat_models import ChatOpenAI
from langchain.utilities import SerpAPIWrapper
from langchain.agents import initialize_agent

In [ ]:
import operator
from itertools import product, permutations
from pydantic import BaseModel, Field
from typing import List


def mydiv(n, d):
    return n / d if d != 0 else 9999999


syms = [operator.add, operator.sub, operator.mul, mydiv]
op = {sym: ch for sym, ch in zip(syms, "+-*/")}


class Solve24Input(BaseModel):
    question: List[int]


def solve24(input_data: Solve24Input) -> str:
    """Solve the 4 numbers to sum 24 problem"""
    print(f"type: {type(input_data)}, value: {input_data}")
    input_model = Solve24Input(question=input_data)
    nums = input_model.question
    for x, y, z in product(syms, repeat=3):
        for a, b, c, d in permutations(nums):
            if round(x(y(a, b), z(c, d)), 5) == 24:
                return f"({a} {op[y]} {b}) {op[x]} ({c} {op[z]} {d})"
            elif round(x(a, y(b, z(c, d))), 5) == 24:
                return f"{a} {op[x]} ({b} {op[y]} ({c} {op[z]} {d}))"
            elif round(x(y(z(c, d), b), a), 5) == 24:
                return f"(({c} {op[z]} {d}) {op[y]} {b}) {op[x]} {a}"
            elif round(x(y(b, z(c, d)), a), 5) == 24:
                return f"({b} {op[y]} ({c} {op[z]} {d})) {op[x]} {a}"
    return "No solution could be found."

In [ ]:
tools = [
    Tool(
        name="Solve_the_sum_24_math_game",
        func=solve24,
        description="useful for when you to solve the make 24 problem using 4 numbers",
        args_schema=Solve24Input,
    ),
]

In [ ]:
tools[0]

In [ ]:
history = ChatMessageHistory()
memory = ConversationSummaryMemory(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=0),
    chat_memory=history,
    return_messages=True,
)
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True,
    memory=memory,
)

In [ ]:
agent_chain.run(
    input="Given the numbers 3, 9, 5, 4 use all 4 numbers and + - × / to make 24"
)

In [ ]:
history = ChatMessageHistory()
history.add_user_message("hi")
history.add_ai_message("hi there!")
memory = ConversationSummaryMemory(
    llm=ChatOpenAI(model_name="gpt-3.5-turbo-0613", temperature=0),
    chat_memory=history,
    return_messages=True,
)
llm = ChatOpenAI(temperature=0)
agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.OPENAI_FUNCTIONS,
    verbose=True,
    memory=memory,
)

In [ ]:
agent_chain.run(
    input="Given the numbers 3, 9, 5, 4 use all 4 numbers and + - × / to make 24"
)